In [4]:
import json

import numpy as np
np.set_printoptions(precision=2)
                 
import pandas as pd

from IPython.display import display

XLSX = '../data/2011-2020-Social-Progress-Index.xlsx'

In [5]:
def spi_hierarchy():
    'Excel の最後のシートに記載されているSPI指標の定義から階層構造を構成する'

    def rec(defs, keys):
        indices = list(defs.keys())[:3]
        dimensions = dict()
        for _, row in defs.iterrows():
            if pd.notna(row).all():
                dimension, component, name = row['Dimension'], row['Component'], row['Indicator name']
                if not dimension in dimensions: dimensions[dimension] = dict()
                components = dimensions[dimension]
                if not component in components: components[component] = []
                components[component].append(name)
        return dimensions

    return rec(pd.read_excel(XLSX, sheet_name='Definitions', skiprows=[0]),
               'Region Name, Sub-region Name, Intermediate Region Name, Country or Area'.split(', '))

In [7]:
IDXTREE = spi_hierarchy()

INDICES1 = list(IDXTREE.keys())
INDICES2 = []
for idx in INDICES1: INDICES2 += list(IDXTREE[idx].keys())
    
SPI = pd.read_excel(XLSX, sheet_name='Data_SPI_2011-2020', skiprows=2)
META = list(SPI.keys())[:6]
SPI = SPI[META + INDICES1 + INDICES2]
SPI = SPI[(SPI['SPI year'] == 2020) & (SPI['SPI Rank'].notna())]

data = SPI.iloc[:, 6:]
data.shape

(163, 15)

# パッケージの読み込み

統計、可視化、色彩科学関連のパッケージを読み込み

In [8]:
# Statistics

# Visualization
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
from bokeh.layouts import column
from bokeh.colors import RGB
from bokeh import palettes

output_notebook()

Loading BokehJS ...

# 可視化

主成分分析 (PCA) の第一、第二主成分をそれぞれ X-, Y-軸として可視化する。

In [29]:
from sklearn.decomposition import PCA

pca = PCA()
XY = pca.fit_transform(data)[:, :2]
SPI['x'], SPI['y'] = XY[:, 0], XY[:, 1]

TOOLTIPS = [ ('Country:', '@Country'), ('Label:', '@labels') ]

from bokeh.plotting import figure, show

def scatter(source, **args):
    p = figure(plot_width=400, plot_height=400, tooltips=TOOLTIPS)
    p.circle(source=source, radius=1.5, line_color=None, **args)
    return p

In [30]:
show(scatter(ColumnDataSource(pd.DataFrame(SPI, columns=['x', 'y', 'Country']))))

# Clustering ($k$-Means)

- データを PCA で次元削減して可視化してみる。

- $k$-Means クラスタリングを施し、各クラスタを着色した。クラスタの色はクラスタの centroid の座標に該当する座標と LCH 色空間を対応づけて決定した。

In [31]:
def kmeans(k):
    from sklearn.cluster import KMeans

    model = KMeans(n_clusters=k, random_state=10).fit(data)
    labels = model.labels_
    
    from sklearn.metrics import silhouette_score
    return { 'k': k, 'labels': labels, 'silhouette': silhouette_score(data, labels)}

## 平均シルエット係数を用いた評価

[Silhouette Coefficient](https://towardsdatascience.com/silhouette-coefficient-validating-clustering-techniques-e976bb81d10c)

平均クラスタ内距離と平均クラスタ間距離に関する評価値。評価値の範囲は [-1, 1]

- 1: クラスタがきれいに分離されている

- 0: かなり疑わしいクラスタリング

- -1: 完全におかしなクラスタリング

In [32]:
clusterings = [kmeans(k) for k in range(3, 15)]

p = figure(plot_height=100, tooltips=[('Silhouette:', '@silhouette')])
p.vbar(source=ColumnDataSource(pd.DataFrame(clusterings)), x='k', top='silhouette', width=0.5)
show(p)

In [33]:
# Colors
from colormath.color_objects import ColorBase, LCHabColor as LCH, sRGBColor as sRGB, LabColor as Lab
from colormath.color_conversions import convert_color

ColorBase.values = ColorBase.get_value_tuple
ColorBase.sRGB = lambda c: convert_color(c, sRGB)
ColorBase.LCH = lambda c: convert_color(c, LCH)
ColorBase.Lab = lambda c: convert_color(c, Lab)

def _color2RGB(c):
    r, g, b = c.sRGB().values()
    return RGB(r * 255, g * 255, b * 255)

ColorBase.RGB = lambda c: _color2RGB(c)

# クラスタ群の色分け

配色にあたって、以下を目標とした

- クラスタの内容の類似性を色に反映させること
- 異なる $k$ に対する配色の対応がとりやすいこと

このために、以下を実施している。

- クラスタの centroid を配色空間 ($\text {CIEL}^*\text {C}^*\text {H}^*$) に次元削減する
- 異なる $k$ ごとの centroid を集めた全 centroid についてまとめて次元削減する

In [14]:
from bokeh.transform import factor_cmap

all_centroids = []

for clustering in clusterings:
    clustering['centroids'] = [data[clustering['labels'] == k].mean() for k in range(clustering['k'])]
    all_centroids = all_centroids + clustering['centroids']
pca_c = PCA()
pca_c.fit(all_centroids)

def pca_cmap(k):
    clustering = None
    for c in clusterings:
        if c['k'] == k:
            clustering = c
            break
    if clustering == None: return
    
    df = pd.DataFrame(SPI, columns=['x', 'y', 'Country'])
    df['labels'] = list(map(lambda x: f'{x:02d}', clustering['labels']))
    
    p = pca_c.transform(clustering['centroids'])[:, :2]
    p /= np.linalg.norm(p, axis=1).max()
    x, y = p[:, 0], p[:, 1]
    c = np.linalg.norm(p, axis=1) * 100
    h = (np.arctan2(y, x) / np.pi + 1) * 180
    palette = [LCH(60, c, h).RGB() for c, h in zip(c, h)]
    clustering['palette'] = palette
    clustering['cmap'] = factor_cmap('labels', palette=palette, factors=list(map(lambda x: f'{x:02d}', range(clustering['k']))))
    
    print(f'silhouette: {clustering["silhouette"]:.3f}')
    plot(ColumnDataSource(df), fill_color=clustering['cmap'], legend_group='labels')

# クラスタ内の特徴の分析


In [25]:
def cluster_attributes(clustering, df):
    keys = df.keys()
    labels = clustering['labels']
    mean = data.mean(axis=0)
    std = data.std(axis=0)

    p = figure(plot_width=clustering['k'] * 100, plot_height=400)

    z_scores = np.ndarray((clustering['k'] + 1, len(keys)))
    for k in range(clustering['k']):
        z_scores[k,:] = (data[labels==k].mean(axis=0) - mean) / std
        p.hbar(y=np.arange(len(keys)), height=0.3, left=k * 3, right=k * 3 + z_scores[k, :],
               color=clustering['palette'][k])

    return p

def show_clusters(k):
    print(f'k = {k}')
    pca_cmap(k)
    cluster_attributes(clusterings[k-3], data)

In [26]:
for k in range(3, 11): show_clusters(k)

k = 3
silhouette: 0.404


k = 4
silhouette: 0.362


k = 5
silhouette: 0.326


k = 6
silhouette: 0.318


k = 7
silhouette: 0.279


k = 8
silhouette: 0.262


k = 9
silhouette: 0.268


k = 10
silhouette: 0.220
